#### 项目背景
通过上述对seaborn库的学习，可以看出seaborn更适合于对多维数据进行可视化分析，特别是分析不同变量之间的相关性关系，这使得它适用于机器学习中的探索性数据分析阶段，通过pairplot成对的绘制所有变量的联合分布，可以直观地看出哪几种变量之间存在着相关关系，为下一步的特征分析和建模指明方向。  

通过学习，我认为可视化分析的应用场景可以大致分为两类：  
1. 对单独一系列数据进行统计分析，适用场景如账单开销或者日程规划等，更适合采用折线图、饼图或玫瑰图来表示，可以使用的库如Altair、Bokeh或pygal等；
2. 对数据集之间的相关关系进行分析，常用于科研绘图或机器学习分析等，一般使用散点图、联合分布或热力图等，Matplotlib和Seaborn更适合做这种工作。

因此，本次项目设计会更关注第二类应用场景，这里我选择了自己准备的一组数据集。先通过爬虫爬取B站23年百大up主的名单以及视频列表，挑选出需要使用的数据存为数据集uplist.csv和videolist.csv。

In [ ]:
# 依赖库
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplot inline

通过pandas加载本地数据集

In [ ]:
uplist_df = pd.read_csv('uplist.csv')
uplist_df

In [ ]:
videolist_df = pd.read_csv('videolist.csv')
videolist_df

先使用pairplot初步分析数据集之间的变量相关关系

In [ ]:
sns.pairplot(uplist_df,kind="reg",x_vars=["follow","view","like","videos"],y_vars=["follow","view","like","videos"])
plt.savefig('uplist_pairplot.png')

In [ ]:
sns.pairplot(videolist_df,kind="reg",x_vars=["view","danmaku","comment"],y_vars=["view","danmaku","comment"])
plt.savefig('videolist_pairplot.png')

接着通过热力图展示数据间的相关关系：

In [ ]:
uplist_cor=uplist_df.corr(method='pearson',numeric_only=True)
sns.heatmap(uplist_cor,cmap="Blues",annot=True,fmt=".2f",linewidth=0.5)
plt.savefig('uplist_heatmap.png')

可以看出，粉丝数与播放量、获赞数有着较强的相关关系，下一步就可以针对这三者进行具体分析：

In [ ]:
g=sns.JointGrid(data=uplist_df,x="view",y="like")
g.plot_joint(sns.kdeplot,shade=True,cbar=True)
g.plot_marginals(sns.boxplot)
plt.savefig('uplist_JointGrid.png')

分析播放量和获赞量的联合分布情况，在大多数情况下，up主的播放量与获赞量成正比。在展示数据时，可以就两者用散点图来表示：

In [ ]:
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
sns.relplot(data=uplist_df,x="view",y="like",hue="frequency",style="frequency")
plt.savefig('uplist_relplot.png')

可以看出，大多数头部up主选择保持双周更或月更的更新频率，更新频率太慢时，播放量和点赞量难以增长，而更新频率更快的up主会获赞量增长低于播放量增长，可以看出这些博主更注重于“以量取胜”的短视频赛道。

接下来对进入百大up主的视频数据进行分析，先通过热力图展示数据之间的相关情况：

In [ ]:
videolist_cor=videolist_df.corr(method='pearson',numeric_only=True)
sns.heatmap(videolist_cor,cmap="Blues",annot=True,fmt=".2f",linewidth=0.5)
plt.savefig('videolist_heatmap.png')

In [ ]:
sns.displot(data=videolist_df,x="timeset",hue="timeset",shrink=0.7)
plt.savefig('videolist_displot.png')

利用直方图对视频长度分段进行分析，可以看出绝大多数人会选择5到20分钟的视频时长，也有小于三分钟的短视频。制作40分钟以上的长视频的内容较少。

In [ ]:
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
sns.relplot(data=videolist_df,x="view",y="danmaku",hue="timeset",style="timeset",col="timeset",col_wrap=3)
plt.savefig('videolist_relplot.png')

可以看出视频长度越长时，弹幕数和观看数的比值就越大，短视频相对播放量较多，但是弹幕数很少

接下来对总体数据进行分析：

In [ ]:
sns.catplot(data=videolist_df,x="timeset",y="view",kind="strip",jitter=0.5,hue="timeset")
plt.savefig('videolist_catplot_view.png')

播放量最高的视频集中在5到20分钟的时间段内，除此以外，3到5分钟的短视频也有较大流量

In [ ]:
sns.catplot(data=videolist_df,x="timeset",y="danmaku",kind="strip",jitter=0.5,hue="timeset")
plt.savefig('videolist_catplot_danmaku.png')

弹幕量最高的则更集中在长视频区间